## LG M50 Single Pulse Parameter Identification

This example presents an experimetal parameter identification method for a two-RC circuit model. The data for this notebook is located within the same directory and was obtained from [[1]](https://github.com/WDWidanage/Simscape-Battery-Library/tree/main/Examples/parameterEstimation_TECMD/Data).


### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [2]:
%pip install --upgrade pip ipywidgets
%pip install pybop[plot] -q
%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: pybop[plot]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [3]:
import pybop
import pybamm
import pandas as pd
from scipy.io import loadmat
import plotly.graph_objects as go

### Importing Data

The data is imported as a dictionary with the following key level:
- ["LGM50_5Ah_Pulse"]
    - ["T0"]
        - ["SoC3"]
            - ["Cell1"]
                - ["data"]

In [32]:
ocp = loadmat("data/LGM50_5Ah_OCV.mat", simplify_cells=True, mat_dtype=False)
pulse_data = loadmat("data/LGM50_5Ah_Pulse.mat", simplify_cells=True, mat_dtype=False)
rate_data = loadmat("data/LGM50_5Ah_RateTest.mat", simplify_cells=True, mat_dtype=False)

# Convert to Dataframes

Next, for the single pulse fit example, we construct a dataframe from the chosen pulse. In this case, we select the data for zero degrees (`T0`) with a state-of-charge of 90% (`SoC9`) and the 19th cell (`Cell19`). 
We apply two filters to the dataframe to ensure the data contains only monotomically increasing time samples without duplicates.

In [31]:
df = pd.DataFrame(pulse_data["LGM50_5Ah_Pulse"]["T0"]["SoC9"]["Cell19"]["data"])
df["ProgTime"] = df["ProgTime"] - df["ProgTime"].min()
df.drop_duplicates(subset=["ProgTime"], inplace=True)

A quick plot of time vs voltage confirms the data looks fine for fitting. In this situation, we would prefer to have additional samples from the relaxation, but as we will show below, PyBOP is still able to identify a parameter set that fits this system.

In [33]:
go.Figure(
    data=go.Scatter(
        x=df["ProgTime"],
        y=df["Voltage"],
    )
)

Next, we construct the OCV function from the import `OCV` data. This is completed with a wrapper method on the `pybamm.Interpolant` function,

In [7]:
def ocv_LGM50(sto):
    name = "OCV"
    x = (ocp["LGM50_5Ah_OCV"]["T25"]["refSoC"].reshape(-1) / 100,)
    y = ocp["LGM50_5Ah_OCV"]["T25"]["meanOCV"].reshape(-1)
    return pybamm.Interpolant(x, y, sto, name)

We can construct the two RC parameter set with initial values as listed. Note, the Initial SOC is shift slightly to better match the zero degree data.

In [8]:
params = pybop.ParameterSet(
    params_dict={
        "chemistry": "ecm",
        "Initial SoC": 0.89,
        "Initial temperature [K]": 25 + 273.15,
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Ambient temperature [K]": 25 + 273.15,
        "Current function [A]": 4.85,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "Cell thermal mass [J/K]": 1000,
        "Cell-jig heat transfer coefficient [W/K]": 10,
        "Jig thermal mass [J/K]": 500,
        "Jig-air heat transfer coefficient [W/K]": 10,
        "Open-circuit voltage [V]": ocv_LGM50,
        "R0 [Ohm]": 0.005,
        "Element-1 initial overpotential [V]": 0,
        "Element-2 initial overpotential [V]": 0,
        "R1 [Ohm]": 0.0001,
        "R2 [Ohm]": 0.0001,
        "C1 [F]": 3000,
        "C2 [F]": 6924,
        "Entropic change [V/K]": 0.0004,
    }
)

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. First, we define the model class with two RC elements.

In [9]:
model = pybop.empirical.Thevenin(
    parameter_set=params.import_parameters(), options={"number of rc elements": 2}
)

In this example, we are going to try to fit all five parameters at once. To do this, we define the `pybop.parameters` as,

In [20]:
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(0.005, 0.0001),
        bounds=[1e-6, 2e-1],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(0.0001, 0.0001),
        bounds=[1e-6, 1],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(0.0001, 0.0001),
        bounds=[1e-6, 1],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(3000, 2500),
        bounds=[0.5, 1e4],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(3000, 2500),
        bounds=[0.5, 1e4],
    ),
]

We can now form the `pybop.Dataset` from the experimental data,

In [27]:
# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": df["ProgTime"].values,
        "Current function [A]": -df["Current"].values,
        "Voltage [V]": df["Voltage"].values,
    }
)

The `FittingProblem` class provides us with a single class that holds all of the objects we need to evaluate our selected `SumSquaredError` cost function. 

In [22]:
problem = pybop.FittingProblem(model, parameters, dataset)
cost = pybop.SumSquaredError(problem)

The cost function can be interrogated manually via the `cost([params])` API. In this example, that would look like the following,

In [28]:
cost([0.01, 0.01, 0.01, 20000, 20000])

1.176754404015497

## Parameter Identification

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the PSO method as it provides global optimisation capability.

In [24]:
optim = pybop.Optimisation(cost, optimiser=pybop.PSO)
optim.set_max_unchanged_iterations(iterations=55, threshold=1e-6)
x, final_cost = optim.run()
print("Initial parameters:", cost.x0)
print("Estimated parameters:", x)

Initial parameters: [5.03983264e-03 1.03830699e-04 1.00999900e-04 1.94436448e+03
 5.99031144e+03]
Estimated parameters: [1.21990630e-05 1.52491447e-02 4.86946639e-02 9.13073369e+02
 5.82307008e-01]


## Plotting and Visualisation

PyBOP provides various plotting utilities to visualize the results of the optimisation.

In [25]:
pybop.quick_plot(x, cost, title="Optimised Comparison");

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [26]:
pybop.plot_convergence(optim)
pybop.plot_parameters(optim);

### Conclusion

This notebook illustrates how to perform circuit model parameter identification using PSO in PyBOP, providing insights into the optimisation process through various visualisations.